In [1]:
import pandas as pd
from datetime import datetime
import numpy as np
import random as py_random
import joblib
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import RFE
import const
from service import fetch_data_from_db
from utils import *
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix

# cria rede neural
import tensorflow as tf

2025-07-03 08:56:03.633267: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-07-03 08:56:05.814644: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-07-03 08:56:06.605529: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-07-03 08:56:08.904773: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [2]:
seed = 41
np.random.seed(seed)
py_random.seed(seed)
tf.random.set_seed(seed)

In [3]:
df = fetch_data_from_db(const.consulta_sql)

In [4]:
df.head()

,profissao,tempoprofissao,renda,tiporesidencia,escolaridade,score,idade,dependentes,estadocivil,produto,valorsolicitado,valortotalbem,classe
0,Cientista de Dados,24,58660.0,Outros,Ens.Médio,MuitoBom,58.0,0,Solteiro,VoyageRoamer,84623.00,350000.00,bom
1,Empresário,21,46557.0,Outros,Ens.Médio,MuitoBom,37.0,2,Víuvo,EcoPrestige,126855.00,500000.00,bom
2,Dentista,13,43939.0,Própria,Ens.Médio,Bom,23.0,0,Casado,DoubleDuty,127151.00,320000.00,ruim
3,Engenheiro,10,37262.0,Própria,Superior,Baixo,36.0,0,Divorciado,AgileXplorer,28767.00,250000.00,bom
4,Contador,6,52606.0,Própria,PósouMais,Justo,27.0,0,Casado,TrailConqueror,199564.00,400000.00,ruim


In [5]:
df["idade"] = df["idade"].astype(int)
df["valorsolicitado"] = df["valorsolicitado"].astype(float)
df["valortotalbem"] = df["valortotalbem"].astype(float)

In [ ]:
substitui_nulos(df)
corrigir_erros_digitacao(df, "profissao", const.profissoes_validas)

In [7]:
#verifica se existem colunas com valores nulos
df.isnull().values.any()

False

In [9]:
#verifca profissoes
df["profissao"].unique()

array(['Cientista de Dados', 'Empresário', 'Dentista', 'Engenheiro',
       'Contador', 'Arquiteto', 'Programador', 'Advogado', 'Médico'],
      dtype=object)

In [ ]:
# query in dataframe
df.query("profissao == 'Programador'").head()

In [14]:
df.query("profissao in ['Programador', 'Arquiteto']").sort_values("profissao", ascending=True)

,profissao,tempoprofissao,renda,tiporesidencia,escolaridade,score,idade,dependentes,estadocivil,produto,valorsolicitado,valortotalbem,classe
5,Arquiteto,37.0,55222.0,Alugada,Ens.Médio,MuitoBom,54.0,2,Víuvo,SpeedFury,271083.0,800000.0,bom
80,Arquiteto,26.0,31394.0,Própria,PósouMais,MuitoBom,53.0,1,Divorciado,ElegantCruise,107035.0,300000.0,ruim
16,Arquiteto,31.0,37159.0,Alugada,Ens.Fundamental,Baixo,48.0,0,Solteiro,AgileXplorer,102127.0,250000.0,ruim
72,Arquiteto,24.0,30627.0,Outros,PósouMais,Justo,34.0,1,Casado,TrailConqueror,41446.0,400000.0,bom
21,Arquiteto,24.0,30627.0,Outros,PósouMais,Justo,35.0,1,Casado,TrailConqueror,41446.0,400000.0,bom
70,Arquiteto,19.0,20189.0,Alugada,Superior,Bom,66.0,0,Divorciado,TrailConqueror,182827.0,400000.0,ruim
63,Arquiteto,36.0,33067.0,Alugada,PósouMais,MuitoBom,31.0,2,Víuvo,WorkMaster,28290.0,280000.0,bom
89,Programador,24.0,19999.0,Outros,Ens.Médio,MuitoBom,34.0,1,Solteiro,DoubleDuty,320000.0,33471.0,ruim
73,Programador,32.0,34502.0,Alugada,Ens.Médio,Baixo,62.0,1,Casado,DoubleDuty,159645.0,320000.0,ruim
68,Programador,23.0,28392.0,Alugada,PósouMais,Bom,55.0,0,Casado,EcoPrestige,111151.0,500000.0,bom


In [15]:
# Trata Outliers
df = tratar_outliers(df, "tempoprofissao", 0, 70)
df = tratar_outliers(df, "idade", 0, 110)

In [16]:
df.describe()

,tempoprofissao,renda,idade,dependentes,valorsolicitado,valortotalbem
count,100.000000,100.000000,100.00000,100.000000,100.000000,100.000000
mean,22.250000,36048.340000,46.15000,0.950000,148794.570000,381175.340000
std,11.337232,13166.015755,14.23708,1.038404,108925.635152,180580.656959
min,0.000000,7814.000000,22.00000,0.000000,28290.000000,33471.000000
25%,11.750000,23807.750000,33.75000,0.000000,76530.000000,295000.000000
50%,23.000000,34502.000000,47.00000,1.000000,127003.000000,325843.500000
75%,32.000000,46631.250000,57.00000,2.000000,171568.750000,400000.000000
max,40.000000,59976.000000,71.00000,4.000000,800000.000000,800000.000000


In [17]:
# Feature Engineering
df["proporcaosolicitadototal"] = (df["valorsolicitado"] / df["valortotalbem"]).astype(
    float
)

DIVISAO DE DADOS Treino e Teste

In [18]:
x = df.drop('classe', axis=1)
y = df['classe']

X_train, X_test, y_train, y_test = train_test_split(
    x, y, test_size=0.2, random_state=seed)

In [20]:
X_train.describe()

,tempoprofissao,renda,idade,dependentes,valorsolicitado,valortotalbem,proporcaosolicitadototal
count,80.000000,80.000000,80.000000,80.000000,80.000000,80.000000,80.000000
mean,21.425000,35742.825000,45.937500,1.050000,151807.562500,389924.125000,0.625828
std,11.543835,13474.949966,14.508264,1.054226,115229.240179,188753.692768,1.264665
min,0.000000,7814.000000,22.000000,0.000000,28290.000000,33471.000000,0.101036
25%,11.000000,23353.750000,31.000000,0.000000,72006.000000,300000.000000,0.211380
50%,20.500000,33959.500000,47.000000,1.000000,128773.000000,340843.500000,0.342282
75%,32.000000,46631.250000,56.250000,2.000000,173156.000000,400000.000000,0.470184
max,40.000000,59976.000000,71.000000,4.000000,800000.000000,800000.000000,9.560515


In [22]:
X_train.describe()

,tempoprofissao,renda,idade,dependentes,valorsolicitado,valortotalbem,proporcaosolicitadototal
count,80.000000,80.000000,80.000000,80.000000,80.000000,80.000000,80.000000
mean,21.425000,35742.825000,45.937500,1.050000,151807.562500,389924.125000,0.625828
std,11.543835,13474.949966,14.508264,1.054226,115229.240179,188753.692768,1.264665
min,0.000000,7814.000000,22.000000,0.000000,28290.000000,33471.000000,0.101036
25%,11.000000,23353.750000,31.000000,0.000000,72006.000000,300000.000000,0.211380
50%,20.500000,33959.500000,47.000000,1.000000,128773.000000,340843.500000,0.342282
75%,32.000000,46631.250000,56.250000,2.000000,173156.000000,400000.000000,0.470184
max,40.000000,59976.000000,71.000000,4.000000,800000.000000,800000.000000,9.560515


In [21]:
y_train.describe()

count      80
unique      2
top       bom
freq       48
Name: classe, dtype: object

NORMALIZACAO DOS DADOS

In [23]:
X_test = save_scalers(
    X_test,
    [
        "tempoprofissao",
        "renda",
        "idade",
        "dependentes",
        "valorsolicitado",
        "valortotalbem",
        "proporcaosolicitadototal",
    ],
)
X_train = save_scalers(
    X_train,
    [
        "tempoprofissao",
        "renda",
        "idade",
        "dependentes",
        "valorsolicitado",
        "valortotalbem",
        "proporcaosolicitadototal",
    ],
)

In [24]:
X_test.head()

,profissao,tempoprofissao,renda,tiporesidencia,escolaridade,score,idade,dependentes,estadocivil,produto,valorsolicitado,valortotalbem,proporcaosolicitadototal
42,Médico,-1.075370,-0.749666,Alugada,Superior,Baixo,0.382360,-0.636146,Casado,SpeedFury,1.688133,3.278995,-0.243710
94,Dentista,-1.585023,1.886050,Alugada,Ens.Fundamental,Baixo,1.070607,1.677113,Víuvo,TrailConqueror,-0.936051,0.388865,-0.402459
37,Médico,1.167107,-1.243455,Própria,Ens.Fundamental,Bom,-0.458831,-0.636146,Víuvo,AgileXplorer,-0.328959,-0.694933,-0.146307
6,Programador,0.555523,-1.130548,Alugada,Ens.Médio,MuitoBom,-0.994135,-0.636146,Divorciado,AgileXplorer,-0.163843,-0.694933,-0.099958
79,Cientista de Dados,1.472900,-1.162952,Outros,PósouMais,MuitoBom,-0.229416,-0.636146,Víuvo,VoyageRoamer,0.089028,0.027599,-0.176511


In [25]:
mapeamento = {"ruim": 0, "bom": 1}
y_train = np.array([mapeamento[item] for item in y_train])
y_test = np.array([mapeamento[item] for item in y_test])
X_train = save_encoders(
    X_train,
    ["profissao", "tiporesidencia", "escolaridade", "score", "estadocivil", "produto"],
)
X_test = save_encoders(
    X_test,
    ["profissao", "tiporesidencia", "escolaridade", "score", "estadocivil", "produto"],
)

In [26]:
y_train

array([1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1,
       0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 0, 0, 1,
       1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 0, 0, 1,
       0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1])

In [27]:
X_train

,profissao,tempoprofissao,renda,tiporesidencia,escolaridade,score,idade,dependentes,estadocivil,produto,valorsolicitado,valortotalbem,proporcaosolicitadototal
86,7,1.270545,-0.173693,0,0,3,0.489863,0.906821,3,7,-0.765741,-0.586042,-0.315746
48,3,-1.257469,1.592340,0,1,0,-1.590970,0.906821,0,4,0.172151,2.186252,-0.327378
49,5,0.921853,-0.352999,0,1,0,-0.411832,0.906821,3,2,-0.569063,0.586851,-0.360088
99,4,-0.298567,-0.779049,1,1,3,0.004335,0.906821,0,3,-0.033907,-0.479416,-0.105626
41,5,1.009026,0.252058,2,1,3,-1.313526,0.906821,1,3,-0.713220,-0.479416,-0.311944
...,...,...,...,...,...,...,...,...,...,...,...,...,...
65,4,0.050124,-0.291687,1,3,3,-1.660331,-1.002275,3,7,-0.557483,-0.586042,-0.247977
80,1,0.398816,-0.324770,2,2,3,0.489863,-0.047727,1,3,-0.391004,-0.479416,-0.214082
12,8,0.573162,0.003896,2,1,1,1.530279,-1.002275,0,7,-0.182492,-0.586042,-0.125952
35,5,1.096199,0.200753,1,2,2,-1.174804,-0.047727,0,1,-0.011777,-0.372789,-0.123847


Verifica quais colunas serao selecionadas para o modelo

In [29]:
model = RandomForestClassifier()
# Instancia o RFE
selector = RFE(model, n_features_to_select=10, step=1)
selector = selector.fit(X_train, y_train)
# Transforma os dados
X_train = selector.transform(X_train)
X_test = selector.transform(X_test)
joblib.dump(selector, './objects/selector.joblib')

['./objects/selector.joblib']

In [30]:
print("Verifica quais colunas serao selecionadas para o modelo")
print(selector.support_)
print(selector.ranking_)

Verifica quais colunas serao selecionadas para o modelo
[ True  True  True  True False False  True False  True  True  True  True
  True]
[1 1 1 1 4 2 1 3 1 1 1 1 1]
